In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
from pylorentz import Momentum4
import itertools

from matplotlib import pyplot as plt

top_mass = 173000

In [42]:
y_vars = ['y1_pt', 'y1_eta', 'y1_phi', 'y1_E', 'y2_pt', 'y2_eta', 'y2_phi', 'y2_E']
jet_vars = ['jet%d_pt','jet%d_eta','jet%d_phi', 'jet%d_E','jet%d_DLR1'] 
labels = []
for x in range(0,11):
    for var in jet_vars:
        labels.append(var %x)
labels = y_vars + labels

In [43]:
signal = pd.read_csv('yy_jj.csv', names=labels)

In [44]:
signal.head(5)

,y1_pt,y1_eta,y1_phi,y1_E,y2_pt,y2_eta,y2_phi,y2_E,jet0_pt,jet0_eta,...,jet9_pt,jet9_eta,jet9_phi,jet9_E,jet9_DLR1,jet10_pt,jet10_eta,jet10_phi,jet10_E,jet10_DLR1
0,45689.4,-2.222350,2.873500,213308.0,34327.6,-0.185603,-0.518296,34920.5,76765.1,1.601680,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79195.1,-1.366260,-0.786426,165349.0,42374.6,-0.514500,2.606080,48107.9,126523.0,-1.339480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,138646.0,-1.774530,3.104360,420587.0,41397.8,-1.735490,0.729734,121048.0,520327.0,-1.194170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103315.0,-0.640509,1.361080,125242.0,48145.5,-1.718220,-2.896020,138515.0,83354.4,-0.663305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85188.3,1.764420,-0.659135,255968.0,51800.2,1.062680,2.618730,83907.2,46918.6,0.996610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
signal_extended = signal.copy()

In [46]:
signal_extended["num_of_jets"] = [int((len(row[row.notna()]) - 8)/5) for i, row in signal.iterrows()]

In [47]:
signal_extended["num_of_jets"]

0          6
1          4
2          5
3          4
4          4
          ..
1780125    5
1780126    5
1780127    4
1780128    4
1780129    5
Name: num_of_jets, Length: 1780130, dtype: int64

In [48]:
#number of jets that have pt <30GeV
num_jets_low = []
for i, row in signal_extended.iterrows():
    pt_lower_than_30 = 0
    jet_btag = 0
    for j in range(int(row["num_of_jets"])):
        if (row['jet%d_pt' %j] < 30000):
            pt_lower_than_30 += 1

    num_jets_low.append(pt_lower_than_30)


In [49]:
signal_extended["num_jets_low"] = num_jets_low

In [50]:
signal_output = []

def sort_jets(jet):
    return jet[1]

In [51]:
for i, row in signal_extended.iterrows():
    #filter out events with less than 4 jets
    if (row["num_of_jets"] - row ["num_jets_low"] <4):
        continue
    
    jets = []
    for index_of_jet in range(int(row["num_of_jets"])):
        if(row["jet%d_pt" %index_of_jet] >= 30000):
            jets.append([index_of_jet, row['jet%d_pt' %index_of_jet],row['jet%d_eta' %index_of_jet], row['jet%d_phi' %index_of_jet], row['jet%d_E' %index_of_jet], row['jet%d_DLR1' %index_of_jet]])
        
    jets.sort(key=sort_jets, reverse= True)
    jets = jets[0:4]

    #condition: keep events with at least one btagged jets
    DLR1s = [jet[-1] for jet in jets]
    if (max(DLR1s) < 2.2):
        continue

    jets_string = jets[0][1:] + jets[1][1:] + jets[2][1:] + jets[3][1:]
    output = []

    for var in y_vars:
        output.append(row[var])

    output = output + jets_string


    signal_output.append(output)









In [52]:
labels2 = []
for x in range(0,4):
    for var in jet_vars:
        labels2.append(var %x)
labels2 = y_vars + labels2

In [53]:
signal_output = pd.DataFrame(signal_output, columns=labels2)

In [54]:
signal_output.head(10)

,y1_pt,y1_eta,y1_phi,y1_E,y2_pt,y2_eta,y2_phi,y2_E,jet0_pt,jet0_eta,...,jet2_pt,jet2_eta,jet2_phi,jet2_E,jet2_DLR1,jet3_pt,jet3_eta,jet3_phi,jet3_E,jet3_DLR1
0,79746.2,-0.392429,2.446330,85965.9,31514.5,-1.718720,-0.717049,90709.7,51651.4,-0.630143,...,36272.9,-0.719251,0.309633,46557.2,-0.539798,31591.8,2.366060,0.071863,169877.0,6.402230
1,136525.0,0.944705,-3.013460,202115.0,49557.9,1.331160,-1.641000,100345.0,151514.0,1.214000,...,103772.0,0.237008,1.376640,107000.0,-4.493890,77253.6,-1.557520,0.761250,191669.0,-3.711870
2,86707.8,0.600250,3.047020,102803.0,79075.2,0.253099,-1.825550,81621.4,136599.0,0.350951,...,50878.6,-2.513780,1.094450,316374.0,-2.530900,35322.4,-0.861525,0.189441,49641.8,2.274870
3,107118.0,-0.571035,-0.569958,125062.0,51644.8,-0.544625,-2.425810,59495.3,221134.0,1.276610,...,41746.0,0.047322,-0.992804,42148.2,3.116970,34905.4,0.578800,0.475945,41239.4,-0.999947
4,103949.0,0.474513,1.977270,115873.0,27256.7,1.215770,-0.377037,50007.5,91406.3,1.680130,...,40432.0,3.422330,0.754473,620117.0,-2.195790,39644.0,0.179441,-2.114970,40719.0,-4.593080
5,111473.0,0.826362,-0.899193,151750.0,41619.4,0.199046,2.082420,42446.6,71275.7,0.253533,...,32254.0,4.063000,2.345060,938051.0,-1.608660,31605.7,2.181650,3.050870,141996.0,2.583910
6,140345.0,0.356950,-1.102100,149381.0,81599.4,1.647270,-1.541970,219722.0,149956.0,-0.539918,...,53031.2,-1.265440,1.958290,101809.0,-1.582200,36452.6,2.147180,2.122050,158309.0,-3.152650
7,113041.0,1.584410,-1.871220,287210.0,41177.5,1.844720,1.681540,133506.0,65155.5,1.328770,...,47011.1,2.051370,0.059000,185969.0,9.575170,30806.4,0.350397,2.308330,32886.8,0.090978
8,64019.9,-1.269320,0.483424,122901.0,33291.2,0.616757,-1.516990,39826.3,50593.9,1.846330,...,37012.8,1.955690,-3.052500,133612.0,7.189140,33146.8,1.587510,0.211848,84646.2,4.147860
9,67642.8,-0.838547,0.065199,92851.3,46169.2,0.257522,-3.121460,47708.5,77853.1,1.315680,...,39844.6,1.959460,1.608320,144298.0,-1.325110,31944.6,2.506000,0.229785,197151.0,2.574660


In [55]:
signal_output.to_csv("background_filtered.csv", index= False)